Решение задачи NER(маркировка слов в соответствии с классами) на базе новостей

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, GRU, Embedding, Bidirectional, Input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from google.colab import drive

In [ ]:
import gdown                                      # Подключим функцию gdown
gdown.download('https://storage.yandexcloud.net/aiueducation/Content/v%200.1/news2.xml', None, quiet=True)      # Скачивание файла

'news2.xml'

In [ ]:
def insert_space(text, index):                          #Объявим функцию введения пробелов. Аргументы-сам текст и индекс символа
  return text[:index] + ' ' + text[index:]              #рядом с которым нужно вставить пробел

In [ ]:
color_meaning = {'00ffff':'личность', 'ffff00':'локация', '00ff00':'организация',
                 'ff00ff':'дата', 'd9d9d9':'национальность', '8e7cc3':'звание',
                 'green':'организация','cyan':'личность','yellow':'локация','magenta':'дата'} 
                                                                                #Создаем словарь соответствий цвет-значение   

In [ ]:
color_dic = {'личность':0,'локация':1,'организация':2,
             'дата':3,'национальность':4,'звание':5}                            #Создаем словарь соответствия чисел и значений

In [ ]:
#названия необходимых тегов
r = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}r'           #тэг r(run фрагмент) текста
rpr = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}rPr'       #тэг rpr(тэг свойств текста runProperties)
t = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}t'           #тэг t(текст)
shd = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}shd'       #тэг shading(заливка ячейки)
fill = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}fill'     #тэг заполнения цветом 
highlight = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}highlight' #тэг выделения background'а текста 
val = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}val'       #тэг значений

In [ ]:
import xml.etree.ElementTree as ET  
#парсим База_новостей_часть_2, но из файла .docx мы вытащили файл document.xml и работаем с ним
tree = ET.parse('news2.xml')                #открываем xml файл
root_elem = tree.getroot()                                                      #получим корневой элемент
body = root_elem[0]                                                             #Возьмем его первый элемент
ps2 = list(iter(body))[:1817]                                                   #получаем список всех новостей

In [ ]:
ps2

[<Element '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}p' at 0x7f8556e77ad0>,
 <Element '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}p' at 0x7f8556e914d0>,
 <Element '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}p' at 0x7f8556ea1230>,
 <Element '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}p' at 0x7f8556e400b0>,
 <Element '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}p' at 0x7f8556e59770>,
 <Element '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}p' at 0x7f8556e66bf0>,
 <Element '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}p' at 0x7f8556e0ad10>,
 <Element '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}p' at 0x7f8556dafc50>,
 <Element '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}p' at 0x7f8556db8bf0>,
 <Element '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}p' at 0x7f8556ddce90>,
 <Element '{http://schemas.openxmlformat

In [ ]:
print(len(ps2))
colors2 = set()                                                                 #Подготовим множество для записи уникальных цветов

1817


In [ ]:
sequences = []                                                                  #Подготовим пустой список последовательностей
tags = []                                                                       #Подготовим пустой список тэгов
zero_symbol = 'o'                                                               #Введем нулевой символ

Парсинг данных

In [ ]:
for i, paragraph in enumerate(ps2):                                             #проходим по всем новостям
    phrases_list = paragraph.findall(r)                                         #находим все фразы (часть новости)

    for phrase in phrases_list:                                                 #Проходим по всем фразам
        words = []                                                              #Создаем пустой список слов внутри одной фразы
        y_list = []                                                             #Категории для слов
        text = phrase.find(t).text                                              #Получаем текст, содержащийся во фразе

        # выделение знаков препинания

        len_text = len(text)                                                    #Определяем длину фразы
        if text.find('.') > -1:                                                 #Ищет на каком месте стоит точка и если она есть, то
          text_dot = text                                                       #записывает текст в новую переменную text_dot
          for j in reversed(range(len_text)):                                   #Теперь идем в обратном направлении от начала строки
            if text[j]=='.':                                                    #Если элемент == точка
              # игнорируем инициалы и сокращения
              if not ( (j > 2 and  text[j-2]=='.') or (j < (len_text - 2) and text[j+2]=='.') or (j > 2 and  text[j-2]==' ') or \
                      (j > 2 and  text[j-3]==' ') ):
                text_dot = insert_space(text_dot, j)                            #Вставим пробел рядом с точкой
        else:
          text_dot = text                                                       #В противном случае запишем текст в text_dot

        len_text = len(text_dot)                                                #Узнаем длину текста
                
        if text_dot.find(',') > -1:                                             #Если нашли запятую, то
          text_comma = text_dot
          for j in reversed(range(len_text)):
            if text_dot[j]==',':
              if not (j < (len_text - 1) and text_dot[j+1].isdigit()): # число через запятую
                text_comma = insert_space(text_comma, j)                        #Ставим пробел рядом с запятой
        else:
          text_comma = text_dot

        text_comma = text_comma.replace('!', ' !')                              #Заменить '!' на 'пробел+!'
        text_comma = text_comma.replace('?', ' ?')
        text_comma = text_comma.replace('(', '( ')
        text_comma = text_comma.replace(')', ' )')
        text_comma = text_comma.replace('[', '[ ')
        text_comma = text_comma.replace(']', ' ]')
        text_comma = text_comma.replace('{', '{ ')
        text_comma = text_comma.replace('}', ' }')
        text_comma = text_comma.replace('"', ' " ')
        text_comma = text_comma.replace('«', '« ')
        text_comma = text_comma.replace('»', ' »')
        text_comma = text_comma.replace(':', ' :')
        text_comma = text_comma.replace(';', ' ;')
        
        text_comma = text_comma.replace('  ', ' ')                              #Заменить двойной пробел на прбел

        text_comma = text_comma.strip()                                         

        if (len(text_comma)>0):                                                 #Если в тексте есть что-то
          text_comma = text_comma.lower()                                       #Понизим регистр букв

          style = phrase.find(rpr)                                              #Получаем стили фразы
          
          if style.find(shd) is not None:                                       #если размечали через заливку
              color = style.find(shd).attrib[fill]                              #получаем значение цвета заливки
          elif style.find(highlight) is not None:                               #если размечали через хайлайт
              color = style.find(highlight).attrib[val]                         #получаем значение цвета хайлайта
          else:
              color = 'white'           #иных вариантов выделения в word нет, значит эта фраза не выделена (белый цвет)
          color = color.lower()                                                 #переводим строковое значение цвета в нижний регистр
          meaning = color_meaning[color] if color in color_meaning else ''      #Если есть цвет в словаре цвет-значение, 
                                                                                #то получаем значение. 
                                                                                #В противном случае у фразы значение не было выделено
          
          colors2.add(color) # добавляем цвет в словарь всех встреченных цветов, если нужно проанализировать их
          
          # print(text_comma, '[{0}]'.format(meaning))

          words = text_comma.split()
          k = len(words)
          if meaning in color_dic:                                              #если во фразе присутствует какая-то выделяемая сущность
            for j in range(k):
              y_list.append(meaning)                                            #получаем индекс позиции, соответствующей какой-то 
                                                                                #семантической(смысловой) окраске и устанавливаем по 
                                                                                #этому индексу абсолютное значение категории
          else:
            for j in range(k):                                                  #В противном случае обозначить как пустой символ
              y_list.append(zero_symbol)

          print(words, ' : ', y_list)                                           #Выведем слова и соответствующие им значения 

          # заполняем глобальный массив 
          for j in range(k):
            sequences.append(words[j])                                          #В список последовательностей заносим слова
            tags.append(y_list[j])                                              #В список тэгов заносим значения

Выходные данные были обрезаны до нескольких последних строк (5000).
['боливии']  :  ['локация']
['.', '"', 'боливийская', 'сторона', 'выразила', 'желание', 'обсудить', 'ввоз', 'на', 'территори']  :  ['o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o']
['ю']  :  ['o']
['российской', 'федерации']  :  ['локация', 'локация']
['мясной', 'продукции', ',', 'в', 'частности', 'говядины', ',', 'и', 'продемонстрировала', 'презентацию', 'касательно', 'эпизоотической', 'ситуации', 'в', 'стране', 'и', 'объема', 'поставок', 'животноводческой', 'продукции…', 'также', 'боливийские', 'коллеги', 'подняли', 'вопрос', 'о', 'поставках', 'растениеводческой']  :  ['o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o']
['продукции', 'в']  :  ['o', 'o']
['россию']  :  ['локация']
['–', 'какао-бобов', ',', 'кофе', 'и', 'каштанов', '"', ',', '—', 'говорится', 'в', 'сообщении', '.', 'отмечается', ',', 'что', 'разрешение', 'на', 'пос

In [ ]:
len(sequences)
 

217981

In [ ]:
tags[0:10]

['звание',
 'o',
 'личность',
 'личность',
 'o',
 'локация',
 'локация',
 'o',
 'o',
 'локация']

In [ ]:
sequences[0:50]

['полковник',
 'запаса',
 'марек',
 'вжосек',
 'назвал',
 'калининградскую',
 'область',
 'военным',
 'пугалом',
 'россии',
 'из-за',
 'количества',
 'размещенных',
 'там',
 'вооружений',
 '.',
 'военный',
 'отметил',
 ',',
 'что',
 'в',
 'последнее',
 'время',
 'москва',
 'наращивает',
 'количество',
 'техники',
 'у',
 'западных',
 'границ',
 'государства',
 '.',
 'так',
 ',',
 'в',
 'начале',
 'этой',
 'недели',
 'противовоздушную',
 'оборону',
 'в',
 'калининградской',
 'области',
 'усилили',
 'полком',
 'с-400',
 '"',
 'триумф',
 '"',
 '.']

In [ ]:
sequencesList = []                                                                 #Список предложений               
tagsList = []                                                                      #Список тэгов
sentence = []                                                                   #Список слов в одном предложении  
tagence = []                                                                    #Список тэгов в предложении
max_sentence_len = 0                                                            #Максимальная длина предложения
current_sentence_len = 0                                                        #Текущая длина предложения
for i in range(len(sequences)):                                                 #Пройдемся по всем словам
  sentence.append(sequences[i])                                                 #Добавить в предложение i-й элемент списка слов 
  tagence.append(tags[i])                                                       #Добавить в тэги предложений реальных эначений тэгов
  current_sentence_len += 1                                                     #Увеличить на 1 текущую длину предложения
  if sequences[i] == '.':                                                       #Если элемент это точка
    sequencesList.append(sentence)                                                 #Запишем в предложение
    tagsList.append(tagence)                                                       #Запишем в список тэгов
    sentence = []                                                               #Обнулим список слов в предложении
    tagence = []                                                                #Обнулим список тэгов в списке тэгов
    max_sentence_len = current_sentence_len if current_sentence_len > max_sentence_len else max_sentence_len   #Если текущее
                                                                                #предложение длиннее максимального, принимаем его  
                                                                                #как максимальное  
    current_sentence_len = 0                                                    #Текуще предложение обнулим  
print('max_sentence_len =',max_sentence_len)                                    #Запишем максимальную длину предложения

max_sentence_len = 148


In [ ]:
print(len(sequencesList),sequencesList[0])

9359 ['полковник', 'запаса', 'марек', 'вжосек', 'назвал', 'калининградскую', 'область', 'военным', 'пугалом', 'россии', 'из-за', 'количества', 'размещенных', 'там', 'вооружений', '.']


In [ ]:
MedLen=len(sequences)/len(sequencesList)
print(MedLen)

23.291056736830857


In [ ]:
len(color_dic)+1

6

In [ ]:
sequences2 = [' '.join(sequence) for sequence in sequencesList]
tags2 = [' '.join(sequence) for sequence in tagsList]

In [ ]:
tagsList.nunique()

AttributeError: ignored

Токенизация последовательностей

In [ ]:
num_words = 5000
sent_len = 30
tokenizer = Tokenizer(num_words)
tag_tokenizer = Tokenizer(len(color_dic)+1, filters=' ')

In [ ]:
tokenizer.fit_on_texts(sequences2)

In [ ]:
X = tokenizer.texts_to_sequences(sequences2)
X = pad_sequences(X, sent_len)
print(X.shape)

(9359, 30)


In [ ]:
tag_tokenizer.fit_on_texts(tags2)
print(tag_tokenizer.index_word)

{1: 'o', 2: 'локация', 3: 'организация', 4: 'личность', 5: 'дата', 6: 'звание', 7: 'национальность'}


In [ ]:
tags[0]

'звание'

In [ ]:
Y = tag_tokenizer.texts_to_sequences(tags2)
Y = pad_sequences(Y, sent_len, value=1)
print(Y.shape)

(9359, 30)


In [ ]:
Y = to_categorical(Y, len(color_dic)+1+1)
np.shape(Y)

(9359, 30, 8)

Модель

In [ ]:
emb_size = 100

input = Input(shape=(None,))
x = Embedding(num_words, emb_size)(input)
x = Bidirectional(LSTM(emb_size, return_sequences=True))(x)
x = LSTM(emb_size * 2, return_sequences=True)(x)
x = Dense(Y.shape[-1], activation='softmax')(x)
#output = CRF()(x)

model = Model(input, x)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X, Y, validation_split=0.2,epochs=15)

Epoch 1/15
234/234 [==============================] - 56s 211ms/step - loss: 0.5172 - accuracy: 0.8873 - val_loss: 0.4146 - val_accuracy: 0.9000
Epoch 2/15
234/234 [==============================] - 48s 206ms/step - loss: 0.4229 - accuracy: 0.8914 - val_loss: 0.4008 - val_accuracy: 0.9002
Epoch 3/15
234/234 [==============================] - 47s 203ms/step - loss: 0.3867 - accuracy: 0.8939 - val_loss: 0.3823 - val_accuracy: 0.9012
Epoch 4/15
234/234 [==============================] - 50s 214ms/step - loss: 0.3434 - accuracy: 0.8989 - val_loss: 0.3629 - val_accuracy: 0.9033
Epoch 5/15
234/234 [==============================] - 49s 209ms/step - loss: 0.3103 - accuracy: 0.9036 - val_loss: 0.3578 - val_accuracy: 0.9015
Epoch 6/15
234/234 [==============================] - 50s 215ms/step - loss: 0.2827 - accuracy: 0.9076 - val_loss: 0.3650 - val_accuracy: 0.9003
Epoch 7/15
234/234 [==============================] - 57s 244ms/step - loss: 0.2616 - accuracy: 0.9104 - val_loss: 0.3780 - val_ac

Пример предсказаний

In [ ]:
pred=model.predict(X[:4])
print("Train set")
print(np.argmax(pred,axis=-1))
print(np.argmax(Y[:4],axis=-1))
pred=model.predict(X[-4:])
print("Test set")
print(np.argmax(pred,axis=-1))
print(np.argmax(Y[-4:],axis=-1))


Train set
[[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4 4 4 1 2 2 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 1 1 1 1 3 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 1]]
[[1 1 1 1 1 1 1 1 1 1 1 1 1 1 6 1 4 4 1 2 2 1 1 2 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 6 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 1 1 1 1 3 1 1]
 [1 1 1 4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 1]]
Test set
[[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 6 1 2 1 1 2 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]]
[[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

In [ ]:
Y[0]

array([[0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 